In [2]:
import json
from mesa import Agent, Model
from mesa.time import RandomActivation
import numpy as np
import geopandas as gpd
import contextily as ctx
from mesa.datacollection import DataCollector
import skmob

# Read jsons

In [3]:
with open('population_dist.json') as f:
  population_dist = json.load(f)

In [4]:
with open('regions_num_dist.json') as f:
  regions_num_dist = json.load(f)

In [5]:
with open('travel_dist.json') as f:
  travel_dist = json.load(f)

# Process distributions

In [6]:
s = np.array(list(population_dist.values())).astype(int).sum()
for key, value in population_dist.items():
    population_dist[key] = int(population_dist[key]) / s

In [7]:
for key, value in travel_dist.items():
    s = travel_dist[key]['true'] + travel_dist[key]['false']
    travel_dist[key]['true'] = travel_dist[key]['true'] / s
    travel_dist[key]['false'] = 1 - travel_dist[key]['true']

In [8]:
for key, value in regions_num_dist.items():
    s = np.array(list(value.values())).astype(int).sum()
    if s != 0:
        for i in range(1, 34):
            regions_num_dist[key][str(i)] = regions_num_dist[key][str(i)] / s

In [9]:
# regions_num_dist

# Regions

In [40]:
regions = gpd.read_file('../../data/raw/EtapII-REJONY_wroclaw.shp')
regions = regions.to_crs(epsg=3857)
regions.head(3)

,NUMBER,NAME,ZST_0_5,ZST_6_15,ZST_16_19,ZST_20_24,ZST_25_44,ZST_45_WE,ZST_WE_I_W,ZST_SUMA,...,ZCZ_25_44,ZCZ_45_WE,ZCZ_WE_I_W,ZCZ_SUMA,GUS_MI,GUS_MI_6_,BD_A_MI_6_,REGON_ZI,BD_A_ZI,geometry
0,22,Komandorska/Swobodna,111.0,200.0,68.0,82.0,670.0,606.0,1412.0,3149.0,...,32.0,11.0,3.0,64.0,3213,3102,3202,1255,1200,"POLYGON ((1896088.508 6638289.682, 1896090.449..."
1,23,Centrum PoÅudniowe,101.0,124.0,49.0,81.0,595.0,547.0,812.0,2309.0,...,55.0,17.0,3.0,95.0,2404,2303,2377,10258,4750,"POLYGON ((1895696.332 6638631.845, 1895609.850..."
2,24,Stysia,154.0,257.0,74.0,123.0,942.0,825.0,1257.0,3632.0,...,34.0,13.0,4.0,137.0,3769,3615,5118,1373,2296,"POLYGON ((1895195.706 6638966.861, 1895023.610..."


# MESA

In [11]:
class Person(Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

        self.start_region = self._sample_region()
        self.current_region = self.start_region
        self.on_the_way = self._sample_status()
        self.end_region = self.start_region
        
        if self.on_the_way:
            self.regions_num_left = self._sample_regions_num() - 1
        else:
            self.regions_num_left = 0

        self.regions_num = self.regions_num_left


    def _sample_region(self):

        sample = np.random.multinomial(1, list(population_dist.values()))
        sample = np.argmax(sample)
        region = list(population_dist.keys())[sample]

        return region

    def _sample_status(self):

        try:
            sample = np.random.multinomial(1, list(travel_dist[str(self.start_region)].values()))
        except KeyError:
            sample = np.random.multinomial(1, np.zeros(2))
        status = not np.argmax(sample)

        return status

    def _sample_regions_num(self):

        try:
            sample = np.random.multinomial(1, list(regions_num_dist[str(self.start_region)].values()))
        except KeyError:
            sample = np.random.multinomial(1, np.zeros(33))
        regions_num = np.argmax(sample) + 1

        return regions_num

    def move(self):

        current_region = regions[regions['NUMBER'] == int(self.current_region)].iloc[0]
        neighbors = regions[~regions.geometry.disjoint(current_region.geometry)]
        neighbors = neighbors['NUMBER'].to_list()

        self.current_region = np.random.choice(neighbors)
        self.regions_num_left = self.regions_num_left - 1

    def step(self):
        
        if self.on_the_way:

            self.move()            

            if self.regions_num_left == 0:
                self.on_the_way = False
                self.end_region = self.current_region

In [12]:
class TrafficModel(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        # Create agents
        for i in range(self.num_agents):
            a = Person(i, self)
            self.schedule.add(a)

        self.datacollector = DataCollector(
            agent_reporters={
                "regions_num": "regions_num",
                "start_region": "start_region",
                "end_region": "end_region"
            }
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [13]:
model = TrafficModel(100)
for i in range(34):
    model.step()

In [33]:
results = model.datacollector.get_agent_vars_dataframe().reset_index()
results = results[results["Step"] == 33]
results = results[['start_region', 'end_region', 'AgentID']].groupby(['start_region', 'end_region']).count().reset_index()
results['start_region'] = results['start_region'].astype(int)
results['end_region'] = results['end_region'].astype(int)
results['AgentID'] = results['AgentID'].astype(int)

In [28]:
results[results['AgentID'] > 1]

,start_region,end_region,AgentID
22,172,172,2
50,293,293,2
63,335,335,2
84,54,52,2


In [38]:
fdf = skmob.FlowDataFrame(
    data=results,
    origin='start_region',
    destination='end_region',
    flow='AgentID',
    tile_id='NUMBER',
    tessellation=regions
)

In [41]:
m = fdf.plot_tessellation(tiles='OpenStreetMap')
fdf.plot_flows(m, flow_color='yellow', tiles='OpenStreetMap', flow_weight=0.5, opacity=0.5)